<a href="https://colab.research.google.com/github/karthebans/Research/blob/main/RNN_Time_series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

In [2]:
def generate_time_series(batch_size,n_steps):
    freq1, freq2, offset1, offset2, = np.random.rand(4,batch_size,1)
    time = np.linspace(0,1,n_steps)
    series = 0.5 * np.sin((time-offset1) * (freq1 * 10 + 10))
    series +=0.2 * np.sin((time - offset2) * (freq2 *20 + 20))
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5)
    return series[..., np.newaxis].astype(np.float32)

In [17]:
n_steps = 50
series = generate_time_series(10000, n_steps +1)
series

array([[[-0.54533035],
        [-0.3996726 ],
        [-0.22606987],
        ...,
        [-0.19724712],
        [-0.40488192],
        [-0.53282756]],

       [[ 0.3056186 ],
        [ 0.4003793 ],
        [ 0.510774  ],
        ...,
        [ 0.18942614],
        [ 0.1407173 ],
        [ 0.0734825 ]],

       [[ 0.11527731],
        [-0.15259919],
        [-0.3420273 ],
        ...,
        [ 0.09055861],
        [-0.02715979],
        [-0.19743623]],

       ...,

       [[ 0.42142615],
        [ 0.24433033],
        [ 0.12374406],
        ...,
        [ 0.28446057],
        [ 0.34511817],
        [ 0.28502357]],

       [[-0.35891268],
        [-0.12081373],
        [ 0.11480688],
        ...,
        [-0.63869727],
        [-0.54450625],
        [-0.26703048]],

       [[ 0.6441406 ],
        [ 0.5917992 ],
        [ 0.4523631 ],
        ...,
        [-0.3469636 ],
        [-0.24242242],
        [-0.31561497]]], dtype=float32)

In [18]:
series[0][0]

array([-0.54533035], dtype=float32)

In [19]:
X_train, y_train = series[:7000, :n_steps], series[:7000, -1]

In [ ]:
X_train

In [21]:
y_train

array([[-0.53282756],
       [ 0.0734825 ],
       [-0.19743623],
       ...,
       [-0.60098857],
       [-0.09373744],
       [ 0.01936147]], dtype=float32)

In [22]:
x_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000,-1]

In [9]:
x_test, y_test = series[9000: , :n_steps], series[9000:,-1]

In [ ]:
x_test

In [ ]:
y_test

In [23]:
import keras

In [24]:
y_pred = x_valid[:,-1]

In [25]:
y_pred

array([[-0.28823906],
       [ 0.2669238 ],
       [-0.26998785],
       ...,
       [ 0.08451651],
       [ 0.04933338],
       [-0.31202528]], dtype=float32)

In [26]:
np.mean(keras.losses.mean_squared_error(y_valid,y_pred))

0.020941718

In [27]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape = [50,1]),
    keras.layers.Dense(1)
])